# 2 Numerical Algebra

## 2.1 Intro to Numerical Root Finding

The golden rule of numerical analysis: We compute only when everything else fails.

In this chapter we want to solve equations using a computer. The goal of equation solving is to find the value of the independent variable which makes the equation true. These are the sorts of equations that you learned to solve in high school algebra and Pre-Calculus. For a very simple example, solve for $ x $ if $ x + 5 = 2x - 3 $. Or for another example, the equation $ x^2 + x = 2x - 7 $ is an equation that could be solved with the quadratic formula. As another example, the equation $ \sin(x) = \frac{\sqrt{2}}{2} $ is an equation which can be solved using some knowledge of trigonometry. The topic of Numerical Root Finding really boils down to approximating the solutions to equations without using all of the by-hand techniques that you learned in high school. The downside to everything that we’re about to do is that our answers are only ever going to be approximations. To see a video introduction to this chapter go to [this link](https://youtu.be/W2yL9IVmv2A).

The fact that we will only ever get approximate answers begs the question: why would we want to do numerical algebra if by-hand techniques exist? The answers are relatively simple:

1. By-hand algebra is often very challenging, quite time-consuming, and error-prone. You will find that the numerical techniques are quite elegant, work very quickly, and require very little overhead to actually implement and verify.
2. Most equations do not lend themselves to by-hand solutions. The techniques that we know from high school algebra solve common, and often quite simplified, problems but when equations arise naturally they are often not nice.

Let’s first take a look at equations in a more abstract way. Consider the equation  
$\ell(x) = r(x)$ where $\ell(x)$ and $r(x)$ stand for left-hand and right-hand expressions respectively. To begin solving this equation we can first rewrite it by subtracting the right-hand side from the left to get  
$$\ell(x) − r(x) = 0.$$

Hence, we can define a function $f(x)$ as $f(x) = \ell(x) − r(x)$ and observe that *every* equation can be written as:  
$$\text{If } f(x) = 0, \text{find } x.$$

This gives us a common language for which to frame all of our numerical algorithms.

For example, if we want to solve the equation  
$$3 \sin(x) + 9 = x^2 − \cos(x)$$ 
then this is the same as solving  
$$(3 \sin(x) + 9) − (x^2 − \cos(x)) = 0.$$ 
We illustrate this idea in Figure 2.1. You should pause and notice that there is no way that you are going to apply by-hand techniques from algebra to solve this equation … an approximate answer is pretty much our only hope.

**Figure 2.1: A Typical Root Finding Problem**

<img src="https://datascienceuwl.github.io/MTH371/figures/Chapter2/Figure_2_1.png" alt="Figure 2.1: A Typical Root Finding Problem">

On the left-hand side of Figure 2.1 we see the solutions to the equation $3\sin(x) + 9 = x^2 - \cos(x)$, and on the right-hand side we see the solutions to the equation

$$
(3\sin(x) + 9) - (x^2 - \cos(x)) = 0.
$$

From the plots it is apparent that the two equations have the same solutions: $x_1 \approx -2.55$ and $x_2 \approx 2.88$. Figure 2.1 should demonstrate what we mean when we say that solving equations of the form $\ell(x) = r(x)$ will give the same answer as solving $f(x) = 0$. Pause for a moment and closely examine the plots to verify this for yourself.

We now have one way to view every equation-solving problem. As we’ll see in this chapter, if $f(x)$ has certain properties then different numerical techniques for solving the equation will apply – and some will be much faster and more accurate than others. The following sections give several different techniques for solving equations of the form $f(x) = 0$. We will start with the simplest techniques to implement and then move to the more powerful techniques that require some ideas from Calculus to understand and analyze. Throughout this chapter we will also work to quantify the amount of error that we make while using these techniques.


## 2.2 The Bisection Method

---

## 2.2.1 Intuition and Implementation

### Exercise 2.1  
A friend tells you that she is thinking of a number between 1 and 100. She will allow you multiple guesses with some feedback for where the mystery number falls. How do you systematically go about guessing the mystery number? Is there an optimal strategy?

For example, the conversation might go like this:

**Sally:** I’m thinking of a number between 1 and 100  
**Joe:** Is it 35?  
**Sally:** No, but the number is between 35 and 100  
**Joe:** Is it 99?  
**Sally:** No, but the number is between 35 and 99  
…

---


### Exercise 2.2  
Now let’s say that Sally has a continuous function that has a root somewhere between  $x = 2$
and  $x = 10.$ Modify your strategy from the number guessing game in the previous problem to narrow down where the root is.

---



### Exercise 2.3  
Was it necessary to say that Sally’s function was continuous? Could your technique work if the function were not continuous?

---



Now let’s get to the math. We’ll start the mathematical discussion with a theorem from Calculus.

### Theorem 2.1 (The Intermediate Value Theorem (IVT))  
If  $f(x)$ is a continuous function on the closed interval $[a,b]$ and $y^*$ lies between $f(a)$ and $f(b)$ then there exists some point $x^* \in [a,b]$ such that $f(x^*) = y^*$.

---



### Exercise 2.4 

Draw a picture of what the intermediate value theorem says graphically.

---



### Exercise 2.5  
If  $y^* = 0$ the Intermediate Value Theorem gives us important information about solving equations. What does it tell us?

---



### Corollary 2.1  
If  $f(x)$ is a continuous function on the closed interval  $[a, b]$ and if $f(a)$ and $f(b)$ have opposite signs, then from the Intermediate Value Theorem, we know that there exists some point  $x^* \in [a, b]$
such that $\underline{\hspace{1in}}$.

---



### Exercise 2.6  

Fill in the blank in the previous corollary and then draw several pictures that indicate why this might be true for continuous functions.

---



### The Intermediate Value Theorem and Its Application  

The Intermediate Value Theorem (IVT) and its corollary are **existence theorems** in the sense that they tell us that some point exists. The annoying thing about mathematical existence theorems is that they typically don’t tell us **how** to find the point that is guaranteed to exist – *annoying*. The method that you developed in **Exercises 2.1 and 2.2** gives one possible way to find the root.

In **Exercises 2.1 and 2.2**, you likely came up with an algorithm such as this:

1. Say we know that the root of a continuous function lies between $x=a$ and $x=b$.
2. Guess that the root is at the midpoint $m = \frac{a + b}{2}$.
3. By using the signs of the function, narrow the interval that contains the root to either $[a,m]$ or $[m,b]$.
4. **Repeat**.

Now we will turn this optimal strategy into **computer code** that will simply play the game for us. But first, we need to pay careful attention to some of the mathematical details.

---



### Exercise 2.7  
Where is the Intermediate Value Theorem used in the root-guessing strategy?

---



### Exercise 2.8  
Why was it important that the function $f(x)$ is continuous when playing this root-guessing game? Provide a few sketches to demonstrate your answer.

---

### Exercise 2.9 (The Bisection Method) 

**Goal:** We want to solve the equation $ f(x) = 0 $ for $ x $ assuming that the solution $ x^* $ is in the interval $[a, b]$.

**The Algorithm:**  Assume that $ f(x) $ is continuous on the closed interval $[a, b]$. To make approximations of the solutions to the equation $ f(x) = 0 $, do the following:

1. **Check to see if** $ f(a) $ **and** $ f(b) $ **have opposite signs.** You can do this by taking the product of $ f(a) $ and $ f(b) $.
   - If $ f(a) $ and $ f(b) $ have different signs then what does the IVT tell you?
   - If $ f(a) $ and $ f(b) $ have the same sign then what does the IVT not tell you? What should you do in this case?
   - Why does the product of $ f(a) $ and $ f(b) $ tell us something about the signs of the two numbers?

2. **Compute the midpoint of the closed interval**, $ m = \frac{a+b}{2} $, and evaluate $ f(m) $.
   - Will $ m $ always be a better guess of the root than $ a $ or $ b $? Why?
   - What should you do here if $ f(m) $ is really close to zero?

3. **Compare the signs of** $ f(a) $ **vs** $ f(m) $ **and** $ f(b) $ **vs** $ f(m) $.
   - What do you do if $ f(a) $ and $ f(m) $ have opposite signs?
   - What do you do if $ f(m) $ and $ f(b) $ have opposite signs?

4. **Repeat steps 2 and 3 and stop when** $ f(m) $ **is close enough to zero.**

---


### Exercise 2.10 

Draw a picture illustrating what the Bisection Method does to approximate the solutions to the equation $f(x)=0.$

---

### Exercise 2.11

We want to write a Python function for the Bisection Method. Instead of jumping straight into the code we should **ALWAYS** write pseudo-code first. It is often helpful to write pseudo-code as comments in your file. Use the template below to complete your pseudo-code.

```python
def Bisection(f , a , b , tol):
    # The input parameters are

    # f is a Python function or a Lambda function
    # a is the lower guess
    # b is the upper guess
    # tol is an optional tolerance for the accuracy of the root

    # if the user doesn't define a tolerance we need
    # code to create a default

    # check that there is a root between a and b
    # if not we should return an error and break the code

    # next calculate the midpoint $m = \frac{a+b}{2}$
    
    # start a while loop
    #   # in the while loop we need an if statement
    #   # if ...
    #   # elif ...
    #   # elif ...

    #   # we should check that the while loop isn't running away

    # end the while loop
    # define and return the root
```

---

### Exercise 2.12 

Now use the pseudo-code as structure to complete a function for the Bisection Method. Also write test code that verifies that your function works properly. Be sure that it can take a Lambda Function as an input along with an initial lower bound, an initial upper bound, and an optional error tolerance. The output should be only 1 single number: the root.



---

### Exercise 2.13
 
Test your Bisection Method code on the following equations.

a. $x^2 - 2 = 0$ on $x \in [0,2]$

b. $\sin(x) + x^2 = 2\ln(x) + 5$ on $x \in [0,5]$ (be careful! make a plot first)

c. $(5 - x)e^x = 5$ on $x \in [0,5]$

---

### 2.2.2 Analysis

After we build any root finding algorithm we need to stop and think about how it will perform on new problems. The questions that we typically have for a root-finding algorithm are:

- Will the algorithm always converge to a solution?
- How fast will the algorithm converge to a solution?
- Are there any pitfalls that we should be aware of when using the algorithm?

---

### Exercise 2.14. 

**Discussion:** What must be true in order to use the bisection method?

---


### Exercise 2.15. 

**Discussion:** Does the bisection method work if the Intermediate Value Theorem does not apply? (Hint: what does it mean for the IVT to "not apply?")

---

### Exercise 2.16. 

If there is a root of a continuous function $f(x)$ between $x=a$ and $x=b$ will the bisection method always be able to find it? Why / why not?

---



Next we'll focus on a deeper mathematical analysis that will allow us to determine exactly how fast the bisection method actually converges to within a pre-set tolerance. Work through the next problem to develop a formula that tells you exactly how many steps the bisection method needs to take in order to stop.

---

### Exercise 2.17. 

Let $f(x)$ be a continuous function on the interval $[a, b]$ and assume that $f(a) \cdot f(b)<0$. A reoccurring theme in Numerical Analysis is to approximate some mathematical thing to within some tolerance. For example, if we want to approximate the solution to the equation $f(x)=0$ to within $\varepsilon$ with the bisection method, we should be able to figure out how many steps it will take to achieve that goal.

1. Let's say that $a=3$ and $b=8$ and $f(a) \cdot f(b)<0$ for some continuous function $f(x)$. The width of this interval is 5 , so if we guess that the root is $m=(3+8) / 2=5.5$ then our error is less than $5 / 2$. In the more general setting, if there is a root of a continuous function in the interval $[a, b]$ then how far off could the midpoint approximation of the root be? In other words, what is the error in using $m=(a+b) / 2$ as the approximation of the root?

2. The bisection method cuts the width of the interval down to a smaller size at every step. As such, the approximation error gets smaller at every step. Fill in the blanks in the following table to see the pattern in how the approximation error changes with each iteration.

| Iteration | Width of Interval | Approximation Error |
| :--- | :--- | :--- |
| 0 | $\|b-a\|$ | $\frac{\|b-a\|}{2}$ |
| 1 | $\frac{\|b-a\|}{2}$ |  |
| 2 | $\frac{\|b-a\|}{2^{2}}$ | $\vdots$ |
| $\vdots$ | $\vdots$ |  |
| $n$ | $\frac{\|b-a\|}{2^{n}}$ |  |

3. Now to the key question: If we want to approximate the solution to the equation $f(x)=0$ to within some tolerance $\varepsilon$ then how many iterations of the bisection method do we need to take?
Hint: Set the $n^{\text {th }}$ approximation error from the table equal to $\varepsilon$. What should you solve for from there?

---



In Exercise 2.17 you actually proved the following theorem.

### Theorem 2.2. (Convergence Rate of the Bisection Method)

If $f(x)$ is a continuous function with a root in the interval $[a, b]$ and if the bisection method
is performed to find the root then:

- The error between the actual root and the approximate root will decrease by a factor of 2 at every iteration.
- If we want the approximate root found by the bisection method to be within a tolerance of $\varepsilon$ then

$$
\frac{|b-a|}{2^{n+1}}=\varepsilon
$$

where $n$ is the number of iterations that it takes to achieve that tolerance.

- Solving for the number of iterations ( $n$ ) we get

$$
n=\log _{2}\left(\frac{|b-a|}{\varepsilon}\right)-1
$$

- Rounding the value of $n$ up to the nearest integer gives the number of iterations necessary to approximate the root to a precision less than $\varepsilon$.



### Exercise 2.18. 

Is it possible for a given function and a given interval that the Bisection Method converges to the root in fewer steps than what you just found in the previous problem? Explain.

---


### Exercise 2.19. 

Create a second version of your Python Bisection Method function that uses a for loop that takes the optimal number of steps to approximate the root to within some tolerance. This should be in contrast to your first version which likely used a while loop to decide when to stop. Is there an advantage to using one of these version of the Bisection Method over the other?

---


The final type of analysis that we should do on the bisection method is to make plots of the error between the approximate solution that the bisection method gives you and the exact solution to the equation. This is a bit of a funny thing! Stop and think about this for a second: if you know the exact solution to the equation then why are you solving it numerically in the first place!?!? However, whenever you build an algorithm you need to test it on problems where you actually do know the answer so that you can can be somewhat sure that it isn't giving you nonsense. Furthermore, analysis like this tells us how fast the algorithm is expected to perform.

From Theorem 2.2 you know that the bisection method cuts the interval in half at every iteration. You proved in Exercise 2.17 that the error given by the bisection method is therefore cut in half at every iteration as well. The following example demonstrate this theorem graphically.

### Example 2.1. 

Let's solve the very simple equation $x^{2}-2=0$ for $x$ to get the solution $x=\sqrt{2}$ with the bisection method. Since we know the exact answer we can compare the exact answer to the value of the midpoint given at each iteration and calculate an absolute error:

$$
\text { Absolute Error }=\mid \text { Approximate Solution }- \text { Exact Solution } \mid .
$$

1. If we plot the absolute error on the vertical axis and the iteration number on the horizontal axis we get Figure 2.2. As expected, the absolute error follows an exponentially decreasing trend. Notice that it isn't a completely smooth curve since we will have some jumps in the accuracy just due to the fact that sometimes the root will be near the midpoint of the interval and sometimes it won't be.

**Figure 2.2: The evolution of the absolute error when solving the equation $x^{2}-2=0$ with the bisection method.**

<img src="https://datascienceuwl.github.io/MTH371/figures/Chapter2/Figure_2_2.png" alt="Figure 2.2: The evolution of the absolute error when solving the equation x^2-2=0 with the bisection method.">


2. Without Theorem 2.2 it would be rather hard to tell what the exact behavior is in the exponential plot above. We know from Theorem 2.2 that the error will divide by 2 at every step, so if we instead plot the base- 2 logarithm of the absolute error against the iteration number we should see a linear trend as shown in Figure 2.3. There will be times later in this course where we won't have a nice theorem like Theorem 2.2 and instead we will need to deduce the relationship from plots like these.

- The trend is linear since logarithms and exponential functions are inverses. Hence, applying a logarithm to an exponential will give a linear function.
- The slope of the resulting linear function should be -1 in this case since we are dividing by 1 power of 2 each iteration. Visually verify that the slope in the plot below follows this trend (the red dashed line in the plot is shown to help you see the slope).

**Figure 2.3: Iteration number vs the base-2 logarithm of the absolute error. Notice the slope of -1 indicating that the error is divided by 1 factor of 2 at each step of the algorithm.**

<img src="https://datascienceuwl.github.io/MTH371/figures/Chapter2/Figure_2_3.png" alt="Figure 2.3: Iteration number vs the base-2 logarithm of the absolute error. Notice the slope of -1 indicating that the error is divided by 1 factor of 2 at each step of the algorithm.">




3. Another plot that numerical analysts use quite frequently for determining how an algorithm is behaving as it progresses is described by the following bullets:

- The horizontal axis is the absolute error at iteration $k$.
- The vertical axis is the absolute error at iteration $k+1$.

See Figure 2.4 below, but this type of plot takes a bit of explaining the first time you see it. Start on the right-hand side of the plot where the error is the largest (this will be where the algorithm starts). The coordinates of the first point are interpreted as:
(absolute error at step 1, absolute error at step 2).
The coordinates of the second point are interpreted as:
(absolute error at step 2, absolute error at step 3).
Etc. Examining the slope of the trend line in this plot shows how we expect the error to progress from step to step. The slope appears to be about 1 in the plot below and the intercept appears to be about -1 . In this case we used a base- 2 logarithm for each axis so we have just empirically shown that

$$
\log _{2}(\text { absolute error at step } k+1) \approx 1 \cdot \log _{2}(\text { absolute error at step } k)-1
$$

Rearranging the algebra a bit we see that this linear relationship turns into

$$
\frac{\text { absolute error at step } k+1}{\text { absolute error at step } k} \approx \frac{1}{2}
$$

(You should stop now and do this algebra.) Rearranging a bit more we get

$$
(\text { absolute error at step } k+1)=\frac{1}{2}(\text { absolute error at step } k)
$$

exactly as expected!! Pause and ponder this result for a second - we just empirically verified the convergence rate for the bisection method just by examining the plot below!! That's what makes these types of plots so powerful!

**Figure 2.4: The base-2 logarithm of the absolute error at iteration $k$ vs the base-2 logarithm of the absolute error at iteration $k+1$.**

<img src="https://datascienceuwl.github.io/MTH371/figures/Chapter2/Figure_2_4.png" alt="Figure 2.4: The base-2 logarithm of the absolute error at iteration k vs the base-2 logarithm of the absolute error at iteration k+1.">


4. The final plot that we will make in analyzing the bisection method is the same as the plot that we just made but with the base-10 logarithm instead. See Figure 2.5. In future algorithms we will not know that the error decreases by a factor of 2 so instead we will just try the base- 10 logarithm. We will be able to extract the exact same information from this plot. The primary advantage of this last plot is that we can see how the order of magnitude (the power of 10) for the error progresses as the algorithm steps forward. Notice that for every order of magnitude iteration $k$ decreases, iteration $k+1$ decreases by one order of magnitude. That is, the slope of the best fit line in Figure 2.5 is approximately 1. Discuss what this means about how the error in the bisection method behaves as the iterations progress.

**Figure 2.5: The base-10 logarithm of the absolute error at iteration $k$ vs the base-10 logarithm of the absolute error at iteration $k+1$.**

<img src="https://datascienceuwl.github.io/MTH371/figures/Chapter2/Figure_2_5.png" alt="Figure 2.5: The base-10 logarithm of the absolute error at iteration k vs the base-10 logarithm of the absolute error at iteration k+1.">

--- 

### Exercise 2.20. 

Carefully read and discuss all of the details of the previous example and plots. Then create plots similar to this example to solve an equation to which you know the exact solution to. You should see the same basic behavior based on the theorem that you proved in Exercise 2.17. If you don't see the same basic behavior then something has gone wrong.

Hints: You will need to create a modified bisection method function which returns all of the iterations instead of just the final root.
If the logarithms of your absolute errors are in a Python list called error then a command like plt.plot (error [:-1], error [1:],'b*') will plot the $(k+1)^{s t}$ absolute error against the $k^{t h}$ absolute error.
If you want the actual slope and intercept of the trend line then you can use `m, b=np.polyfit(error[:-1], error[1:], deg=1)`.

---


### 2.3 The Regula Falsi Method

### 2.3.1 Intuition and Implementation

The bisection method is one of many methods for performing root finding on a continuous function. The next algorithm takes a slightly different approach.

---


### Exercise 2.21. 

In the Bisection Method, we always used the midpoint of the interval as the next approximation of the root of the function $f(x)$ on the interval $[a, b]$. The three pictures in Figure 2.6 show the same function with three different choices for $a$ and $b$. Which one will take fewer Bisection-steps to find the root? Which one will take more steps? Explain your reasoning.
(Note: The root in question is marked with the green star and the initial interval is marked with the red circles.)

**Figure 2.6: In the bisection method you get to choose the starting interval however you like. That choice will make an impact on how fast the algorithm converges to the approximate root.**

<img src="https://datascienceuwl.github.io/MTH371/figures/Chapter2/Figure_2_6.png" alt="Figure 2.6: In the bisection method you get to choose the starting interval however you like. That choice will make an impact on how fast the algorithm converges to the approximate root.">



---

### Exercise 2.22. 

Now let's modify the Bisection Method approach. Instead of always using the midpoint (which as you saw in the previous problem could take a little while to converge) let's draw a line between the endpoints and use the $x$-intercept as the updated guess. If we use this method can we improve the speed of convergence on any of the choices of $a$ and $b$ for this function? Which one will now likely take the fewest steps to converge? Figure 2.7 shows three different starting intervals marked in red with the new guess marked as a black X.

**Figure 2.7: In hopes of improving the bisection method we instead propose that we choose the intersection of a line between the endpoints of the interval and the $x$ axis. The intersection (marked with a black X) would be the next approximation instead of the midpoint of the interval.**

<img src="https://datascienceuwl.github.io/MTH371/figures/Chapter2/Figure_2_7.png" alt="Figure 2.7: In hopes of improving the bisection method we instead propose that we choose the intersection of a line between the endpoints of the interval and the $x$ axis. The intersection (marked with a black X) would be the next approximation instead of the midpoint of the interval.">


---

The algorithm that you played with graphically in the previous problem is known as the Regula Falsi (false position) algorithm. It is really just a minor tweak on the Bisection method. After all, the algorithm is still designed to use the Intermediate Value Theorem and to iteratively zero in on the root of the function on the given interval. This time, instead of picking the midpoint of the interval that contains the root we draw a line between the function values at either end of the interval and then use the intersection of that line with the $x$ axis as the new approximation of the root. As you can see in Figure 2.7 you might actually converge to the approximate root much faster this way (like with the far right plot) or you might gain very little performance (like the far left plot).

---

### Exercise 2.23. (The Regula Falsi Method) 

Assume that $f(x)$ is continuous on the interval $[a, b]$. To make iterative approximations of the solutions to the equation $f(x)=0$, do the following:

1. Check to see if $f(a)$ and $f(b)$ have opposite signs so that the intermediate value theorem guarantees a root on the interval.
2. We want to write the equation of the line connecting the points $(a, f(a))$ and $(b, f(b))$.

- What is the slope of this line?

$$
m=
$$

$\qquad$

- Using the point-slope form of a line, $y-y_{1}=m\left(x-x_{1}\right)$, what is the
equation of the line?

$$
y-\quad=\quad . \quad\left(x-\quad{ }_{\square}\right)
$$

3. Find the $x$ intercept of the linear function that you wrote in the previous step by setting the $y$ to zero and solving for $x$. Call this point $x=c$.

$$
c=
$$

$\qquad$
Hint: The $x$ intercept occurs with $y=0$.
4. Just as we did with the bisection method, compare the signs of $f(a)$ vs $f(c)$ and $f(b)$ vs $f(c)$. Replace one of the endpoints with $c$. Which one do you replace and why?
5. Repeat steps 2-4, and stop when $f(c)$ is close enough to zero.

---



### Exercise 2.24. 

Draw a picture of what the Regula Falsi method does to approximate a root.



---

### Exercise 2.25. 

Give sketches of functions where the Regula Falsi method will perform faster than the Bisection method and visa versa. Justify your thinking with several pictures and be prepared to defend your answers.



---

### Exercise 2.26. 

Create a new Python function called regulafalsi and write comments giving pseudo-code for the Regula-Falsi method. Remember that starting with pseudo-code is always the best way to start your coding. Write comments that give direction to the code that you're about to write. It is a trap to try and write actual code without any pseudo-code to give you a backbone for the function.



----

### Exercise 2.27. 

Use your pseudo-code to create a Python function that implements the Regula Falsi method. Write a test script that verifies that your function works properly. Your function should accept a Python function or a Lambda function as input along with an initial lower bound, an initial upper bound, and an optional error tolerance. The output should be only 1 single number: the approximate root.

